[실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터 - KADX 농식품 빅데이터 거래소](https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_9334c816-5826-4d0e-a8f4-0b9eb47f6f4d/5#!)

## 실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터 
* 주력 상품 카테고리 파악: 어떤 상품 카테고리가 가장 높은 매출을 기록하는지 파악하여 마케팅 전략 수립에 활용할 수 있습니다.
* 고객 세분화: RFM 분석을 통해 고객을 세분화하여 맞춤형 마케팅 전략을 수립할 수 있습니다.
* 구매 패턴 분석: 주차별, 연령별, 제품 카테고리별 구매 패턴을 분석하여 프로모션 시행 시점이나 재고 관리 전략 수립에 활용할 수 있습니다.
* 매출 변동 분석: 주차별, 계절별 매출 변동을 분석하여 매출 예측 모델을 개발하고, 사업 계획 수립에 활용할 수 있습니다.

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df = pd.read_excel("data/실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터.xlsx")
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# 중복데이터 확인
df[df.duplicated()]

In [ ]:
# 결측치 확인
df.isnull().sum()

In [ ]:
# 수치형 변수의 기술 통계 확인
df.describe()

In [ ]:
# 범주형 변수의 기술 통계 확인
df.describe(include="object")

In [ ]:
# 전체 변수의 고유값(유일값)
df.nunique()

In [ ]:
# 구매 대분류별 판매 성과 분석
channel_sales_summary = df.groupby('대분류')[['구매수량', '구매금액']].agg(['sum', 'mean', 'count']).reset_index()
channel_sales_summary

In [ ]:
# 대분류별 매출 비율
total_sales = df['구매금액'].sum()
channel_sales_summary[('매출 비율', 'percent')] = (channel_sales_summary[('구매금액', 'sum')] / total_sales) * 100
channel_sales_summary

In [ ]:
# 대분류별 주요 구매 상품 분석
top_products_by_channel = df.groupby('대분류')['구매상품명'].apply(lambda x: x.value_counts().index[0]).reset_index()
top_products_by_channel.columns = ['대분류', '가장 많이 구매된 상품']
top_products_by_channel

In [ ]:
# 주차별 구매 패턴 분석
weekly_sales_pattern = df.groupby('구매주차')['구매금액'].sum().reset_index()
weekly_sales_pattern

In [ ]:
age_sales = df.groupby('구매자연령')['구매금액'].sum().sort_values(ascending=False)
age_sales

In [ ]:
df['구매일'] = pd.to_datetime(df['구매주차일자'].str.split('~').str[0].str.strip())
daily_sales = df.groupby('구매일')['구매금액'].sum().sort_index()
daily_sales

In [ ]:
(df["구매일"].max() - df['구매일']).dt.days

## RFM 분석

RFM 분석을 통해 고객을 최근성(Recency), 빈도(Frequency), 금액(Monetary) 기준으로 세분화합니다. RFM 분석은 고객의 가치를 평가하고 마케팅 전략을 수립하는 데 유용한 도구입니다.

In [ ]:
def rfm_analysis(df):
    # 최근성(Recency): 가장 최근 구매일
    df['구매일'] = pd.to_datetime(df['구매주차일자'].str.split('~').str[0].str.strip())
    recency = df.groupby('구매자연령')['구매일'].max()
    
    # 빈도(Frequency): 구매 빈도
    frequency = df.groupby('구매자연령')['테이블순번'].count()
    
    # 금액(Monetary): 총 구매 금액
    monetary = df.groupby('구매자연령')['구매금액'].sum()
    
    rfm = pd.concat([recency, frequency, monetary], axis=1)
    rfm.columns = ['Recency', 'Frequency', 'Monetary']
    return rfm

rfm = rfm_analysis(df)
rfm

## 시각화 


시각화는 복잡한 데이터 패턴을 직관적으로 이해하는 데 도움을 줍니다. 예를 들어, Treemap과 Sunburst Chart를 통해 제품 카테고리 간의 상대적 중요도를 쉽게 파악할 수 있으며, 시계열 차트를 통해 매출의 계절성이나 트렌드를 식별할 수 있습니다.

### plotly
* Treemap, Sunburst Chart: 제품 카테고리별 매출 비중을 시각적으로 표현하여 전체적인 매출 구조를 파악합니다.
* 히스토그램: 연령대별, 제품 카테고리별, 구매 주차별 구매 금액 및 수량 분포를 확인하여 데이터의 분포를 시각적으로 파악합니다.
* 산점도: 연령대별 구매 금액과 수량의 관계를 시각적으로 표현하여 연령대별 소비 패턴을 파악합니다.
* 원형 차트: 제품 카테고리별 매출 비중을 원형 차트로 표현하여 전체적인 매출 구조를 파악합니다.
* 선 그래프: 주차별 매출 추이를 시각적으로 표현하여 매출 변동을 파악합니다.


In [ ]:
# Treemap Chart 시각화
px.treemap(
    df,
    path=['대분류', '중분류', '소분류'],  # 계층 구조를 나타내는 열
    values='구매금액',  # 크기를 나타내는 열
    width=900, height=600,
    title='제품 카테고리별 구매 금액 Treemap Chart'
)

In [ ]:
# Sunburst Chart 시각화
px.sunburst(
    df,
    path=['대분류', '중분류', '소분류'],  # 계층 구조를 나타내는 열
    values='구매금액',  # 크기를 나타내는 열
    width=900, height=900,
    title='제품 카테고리별 구매 금액 Sunburst Chart'
)

In [ ]:
# 연령대별 구매 금액 히스토그램
px.histogram(df, x='구매금액', color='구매자연령', nbins=20, title='연령대별 구매 금액 분포')

In [ ]:
# 대분류별 구매 금액 히스토그램
px.histogram(df, x='구매금액', color='대분류', nbins=20, title='제품 대분류별 구매 금액 분포')

In [ ]:
# 구매 주차별 구매 수량 히스토그램
px.histogram(df, x='구매주차', y='구매수량', color='구매주차', title='구매 주차별 구매 수량', histfunc='sum')

In [ ]:
# 연령대별 구매 금액 및 수량
px.scatter( df, x='구매자연령', y='구매금액', size='구매수량', color='구매자연령', title='연령대별 구매 금액 및 수량')

In [ ]:
# 제품 카테고리별 판매 성과
px.pie(
    df,
    names='대분류',
    values='구매금액',
    width=900, height=900,
    title='제품 카테고리별 판매 성과'
)

In [ ]:
# 주차별 매출 추이
weekly_sales_pattern = df.groupby('구매주차')['구매금액'].sum().reset_index()
px.line(
    weekly_sales_pattern,
    x='구매주차',
    y='구매금액',
    title='주차별 매출 추이'
)